In [1]:
"""
Association rule mining
"""
import os
import numpy as np
import pandas as pd

In [2]:
path = os.getcwd()
files = os.listdir(path)

In [71]:
"""

Key ideas to report in the paper
1. General statistics: Number of pipes with defects, most frequent defects 
2. Number of instances of two defects being within the same vicinity, probabilities of these occurrences
3. Support, lift, etc.

"""

'\n\nKey ideas to report in the paper\n1. General statistics: Number of pipes with defects, most frequent defects \n2. Number of instances of two defects being within the same vicinity, probabilities of these occurrences\n3. Support, lift, etc.\n\n'

In [4]:
"""
Helper functions
"""

def change_defect_names(df_cond):
    """ 
    Change the names of defects: e.g., FL => Fracture, TB => Tap
    Takes df_cond as input. 
    """
    df_cond.loc[df_cond['PACP_Code'].str.startswith('F'), 'PACP_Code'] = 'Fracture'
    df_cond.loc[df_cond['PACP_Code'].str.startswith('C'), 'PACP_Code'] = 'Crack'
    df_cond.loc[df_cond['PACP_Code'].str.startswith('T'), 'PACP_Code'] = 'Tap'
    df_cond.loc[df_cond['PACP_Code'].str.startswith('A'), 'PACP_Code'] = 'Manhole'
    df_cond.loc[df_cond['PACP_Code'].str.startswith('J'), 'PACP_Code'] = 'Joint Offset'
    df_cond.loc[df_cond['PACP_Code'].str.startswith('R'), 'PACP_Code'] = 'Root'
    df_cond.loc[df_cond['PACP_Code'].str.startswith('D'), 'PACP_Code'] = 'Deposit'
    df_cond.loc[df_cond['PACP_Code'].str.startswith('B'), 'PACP_Code'] = 'Broken'  
    return df_cond

def delete_rows(df_cond, keep_defects):
    """
    Delete rows where the defects that we want to consider are not present
    """
    df_cond = df_cond[df_cond['PACP_Code'].isin(keep_defects)]
    return df_cond
    

def select_df(df, insp_id, defect_list):
    """
    Helper function used by count_defect_pairs()
    This function only keeps the rows where ['defect1', 'defect2'] are present
    It helps speed up the computation
    """
    df = df[(df['InspectionID'] == insp_id)]
    return  df[df['PACP_Code'].isin(defect_list)]

def delete_inspections_with_no_defects(df_cond, defects):
    """
    Delete those inspections where there aren't any defects at all. 
    By defects we mean fractures, cracks, etc. Not tap and manhole
    """
    insp_ids = np.unique(list(df_cond['InspectionID']))

    keep_ids = [] # Inspection IDs that we want to keep

    for insp_id in insp_ids:
        df_temp = df_cond[(df_cond['InspectionID'] == insp_id)]
        if df_temp['PACP_Code'].isin(defects).sum() > 0:
            keep_ids.append(insp_id)
    
    return df_cond[df_cond['InspectionID'].isin(keep_ids)]

def count_defect_pairs(df_cond, keep_defects, distance_threshold):
    """
    Count the number of defect pairs that are < distance_threshold within each other
    """
    defect_pair_counts = []
    
    for a in range(0, len(keep_defects)): 
        for b in range(a+1, len(keep_defects)):

            pair_count = 0
            defect_pair = [keep_defects[a], keep_defects[b]] # Pair of defects whose no. of occurrences we wish to calculate

            for insp_id in df_cond['InspectionID'].unique():
            
            # Inspection id which we are counting for
                df = select_df(df_cond, insp_id, defect_pair)
                insp_id = list(df['InspectionID'])
                dist = list(df['Distance'])
                defects = list(df['PACP_Code']) # Defects which are selected from a particular inspection

                for i in range (0, len(defects) - 1):
                    for j in range (i+1, len(defects)):
                        defect_1, defect_2 = defects[i], defects[j]
                        if (defect_1 == defect_pair[0] and defect_2 == defect_pair[1]) or (defect_2 == defect_pair[0] and defect_1 == defect_pair[1]):
                            if abs(dist[i] - dist[j]) <= distance_threshold:
                                pair_count = pair_count + 1

            defect_pair_counts.append([keep_defects[a], keep_defects[b], pair_count])
            
    return defect_pair_counts

def calculate_length_of_pipeline(df_cond):
    # Find total length of pipeline
    insps = df_cond['InspectionID'].unique()
    length = 0.0
    for insp in insps:
        df_temp = df_cond[df_cond['InspectionID'] == insp]
        length += df_temp['Distance'].max()
    return length
   

def create_econometric_database(df_insp, cols, defects):
    df_insp = df_insp[cols]

    f, c, t, m, j, r, d, b = [], [], [], [], [], [], [], []

    insp_ids = list(df_insp['InspectionID'])

    for insp_id in insp_ids:

        df_temp = df_cond.loc[df_cond['InspectionID'] == insp_id]

        f.append(len(df_temp[df_temp['PACP_Code'] == 'Fracture']))
        c.append(len(df_temp[df_temp['PACP_Code'] == 'Crack']))
        t.append(len(df_temp[df_temp['PACP_Code'] == 'Tap']))
        m.append(len(df_temp[df_temp['PACP_Code'] == 'Manhole']))
        j.append(len(df_temp[df_temp['PACP_Code'] == 'Joint Offset']))
        r.append(len(df_temp[df_temp['PACP_Code'] == 'Root']))
        d.append(len(df_temp[df_temp['PACP_Code'] == 'Deposit']))
        b.append(len(df_temp[df_temp['PACP_Code'] == 'Broken']))

    df_insp['Fracture'] = f
    df_insp['Crack'] = c
    df_insp['Tap'] = t
    df_insp['Manhole'] = m
    df_insp['Joint Offset'] = j
    df_insp['Root'] = r
    df_insp['Deposit'] = d
    df_insp['Broken'] = b
    
    return df_insp

In [5]:
cluster_size = 5000

# Read the database of condition information from CSV
df_cond = pd.read_csv('Conditions_Combined.csv', sep=',')
df_cond.head();
print("Total number of inspections to begin with are: {}".format(df_cond['InspectionID'].max()))

# Change names of defects
df_cond = change_defect_names(df_cond)

# Keep only the rows which contain these defects. Delete all else
keep_defects = ['Fracture', 'Crack', 'Tap', 'Manhole', 'Joint Offset', 'Root', 'Deposit', 'Broken']
df_cond = delete_rows(df_cond, keep_defects)

# Delete inspections which have no 'defects', i.e, fracture, crack, root, joint offset, deposit, broken
defects = ['Fracture', 'Crack', 'Joint Offset', 'Root', 'Deposit', 'Broken']
# df_cond = delete_inspections_with_no_defects(df_cond, defects)
# print("Length : {}".format(calculate_length_of_pipeline(df_cond)))


# Count the number of inspections that remain after the deletion
# print("\nNumber of inspections which contain defects is: {}".format(df_cond['InspectionID'].nunique()))

# List the number of defects
print("\nNumber of defects is as follows: \n{}".format(df_cond['PACP_Code'].value_counts()))

"""
Counting only Fracture and Tap
Delete all inspections which do not have both Fracture and Tap
"""

keep_defects = ['Joint Offset', 'Fracture']

# Deletes those inspections which don't have defect1 or defect2
df_cond = delete_inspections_with_no_defects(df_cond, [keep_defects[0], keep_defects[1]])

# Count the number of inspections that remain after the deletion
print("\nNumber of inspections with {} and {} is: {}".format(keep_defects[0], keep_defects[1], df_cond['InspectionID'].nunique()))

# List the number of defects
print("\nNumber of defects in these inspections is as follows: \n{}".format(df_cond['PACP_Code'].value_counts()))

# Counting the number of defect clusters within 5 feet
defect_pair_counts = count_defect_pairs(df_cond, keep_defects, cluster_size)

print('\nNumber of ({}, {}) clusters is: {}'.format(keep_defects[0], keep_defects[1], defect_pair_counts[0][2]))

# Count the total length of pipe in this dataframe
length = calculate_length_of_pipeline(df_cond)

print("Length : {}".format(length))

Total number of inspections to begin with are: 8891

Number of defects is as follows: 
Tap             29688
Manhole         17167
Crack            9233
Fracture         8625
Deposit          8287
Root             7943
Broken           2058
Joint Offset     1943
Name: PACP_Code, dtype: int64

Number of inspections with Joint Offset and Fracture is: 2040

Number of defects in these inspections is as follows: 
Tap             10247
Fracture         8625
Crack            7326
Deposit          4171
Root             3983
Manhole          3795
Joint Offset     1943
Broken           1791
Name: PACP_Code, dtype: int64

Number of (Joint Offset, Fracture) clusters is: 8878
Length : 420091.00050309947


In [6]:
num_defect1 = df_cond['PACP_Code'].value_counts()[keep_defects[0]]
num_defect2 = df_cond['PACP_Code'].value_counts()[keep_defects[1]]
print("Number of {} is {}".format(keep_defects[0], num_defect1))
print("Number of {} is {}".format(keep_defects[1], num_defect2))

spacing1 = length/num_defect1
spacing2 = length/num_defect2

print("Length: {}".format(length))
print("Average {} spacing: {} feet".format(keep_defects[0], spacing1))
print("Average {} spacing: {} feet".format(keep_defects[1], spacing2))

Number of Joint Offset is 1943
Number of Fracture is 8625
Length: 420091.00050309947
Average Joint Offset spacing: 216.20741147869248 feet
Average Fracture spacing: 48.7062029568811 feet


In [7]:
# If Taps and Fractures were uniformly distributed, calculate the number of co-occurrences
import numpy as np

defect1 = [[keep_defects[0], i, 1] for i in np.arange(0, length, spacing1)]
defect2 = [[keep_defects[1], i, 1] for i in np.arange(0, length, spacing2)]
df_cond_fake = pd.DataFrame(defect1+defect2, columns = ['PACP_Code', 'Distance', 'InspectionID'])
defect_pair_counts = count_defect_pairs(df_cond_fake, [keep_defects[0], keep_defects[1]], cluster_size)
print('\nNumber of ({}, {}) clusters is: {}'.format(keep_defects[0], keep_defects[1], defect_pair_counts[0][2]))


Number of (Joint Offset, Fracture) clusters is: 396550


In [ ]:
"""
Count number of defect hotspots

TODO: Compare with uiform distribution/Weibull distribution/Try fitting curve

"""

In [5]:
# Count number of defect zones
df_cond = pd.read_csv('Conditions_Combined.csv', sep=',')
df_cond.head();
print("Total number of inspections to begin with are: {}".format(df_cond['InspectionID'].nunique()))

# Change names of defects
df_cond = change_defect_names(df_cond)

# Keep only the rows which contain these defects. Delete all else
keep_defects = ['Fracture', 'Crack', 'Joint Offset', 'Root', 'Deposit', 'Broken']
df_cond = delete_rows(df_cond, keep_defects)

# Delete inspections which have no 'defects', i.e, fracture, crack, root, joint offset, deposit, broken
defects = ['Fracture', 'Crack', 'Joint Offset', 'Root', 'Deposit', 'Broken']
df_cond = delete_inspections_with_no_defects(df_cond, defects)

print("Number of inspections after deletion are: {}".format(df_cond['InspectionID'].nunique()))

Total number of inspections to begin with are: 8891
Number of inspections after deletion are: 4696


In [35]:
thresh = 5

insps = df_cond['InspectionID'].unique()
zones = []

for insp in insps:
    
    df_temp = df_cond[df_cond['InspectionID'] == insp]
    df_temp = df_temp.sort_values(by=["Distance"])
    indices = df_temp.index
    defect_prev, defect_curr = "",""
    dist_prev, dist_curr = 0, 0
    zone_curr = []
    
    for index in indices:
        defect_curr = df_temp.at[index, 'PACP_Code'] # Defect code at current index
        dist_curr = float(df_temp.at[index, 'Distance']) # Distance of defect at current index        
    
        if abs(dist_curr - dist_prev) < thresh:
            zone_curr.append((insp, defect_curr, dist_curr))
        else:           
            zones.append(zone_curr)
            zone_curr = []
            zone_curr.append((insp, defect_curr, dist_curr))
                        
        dist_prev = dist_curr
        defect_prev = defect_curr
        
    zones.append(zone_curr)




In [30]:
zones

[[(4, 'Deposit', 1.0)],
 [(4, 'Deposit', 57.29999924), (4, 'Deposit', 57.29999924)],
 [(4, 'Deposit', 77.09999847)],
 [(4, 'Deposit', 121.90000149999999)],
 [(4, 'Deposit', 129.5)],
 [],
 [(5, 'Deposit', 12.80000019)],
 [(5, 'Deposit', 27.60000038)],
 [(5, 'Deposit', 88.90000153)],
 [(5, 'Deposit', 101.0)],
 [(5, 'Deposit', 122.6999969)],
 [],
 [(6, 'Deposit', 9.600000381000001), (6, 'Deposit', 13.69999981)],
 [(6, 'Fracture', 21.10000038)],
 [(6, 'Deposit', 37.09999847)],
 [(6, 'Fracture', 75.5), (6, 'Broken', 75.5)],
 [(6, 'Deposit', 112.8000031)],
 [(6, 'Deposit', 195.1999969)],
 [(6, 'Deposit', 236.60000609999997)],
 [],
 [],
 [],
 [],
 [],
 [(11, 'Deposit', 10.80000019)],
 [(11, 'Deposit', 127.8000031)]]

In [63]:
"""
Calculate number of colocated defects
"""

colocated = 0
total = 0
max_colocated = 0

for zone in zones:
    if len(zone) > 1:
        colocated += len(zone)
        if len(zone) > max_colocated:
            max_colocated = len(zone)
            max_zone = zone
    total += len(zone)

In [64]:
max_zone

[(3309, 'Root', 77.90000153),
 (3309, 'Root', 80.19999695),
 (3309, 'Joint Offset', 82.69999695),
 (3309, 'Root', 84.30000305),
 (3309, 'Joint Offset', 85.30000305),
 (3309, 'Root', 85.30000305),
 (3309, 'Joint Offset', 85.30000305),
 (3309, 'Joint Offset', 87.80000305),
 (3309, 'Root', 87.80000305),
 (3309, 'Joint Offset', 90.40000153),
 (3309, 'Crack', 91.0),
 (3309, 'Root', 92.59999847),
 (3309, 'Joint Offset', 95.30000305),
 (3309, 'Root', 95.30000305),
 (3309, 'Root', 97.90000153),
 (3309, 'Root', 100.4000015),
 (3309, 'Fracture', 100.9000015),
 (3309, 'Root', 103.0999985),
 (3309, 'Crack', 105.6999969),
 (3309, 'Joint Offset', 105.6999969),
 (3309, 'Root', 107.1999969),
 (3309, 'Joint Offset', 107.40000149999999),
 (3309, 'Joint Offset', 110.3000031),
 (3309, 'Fracture', 110.90000149999999),
 (3309, 'Root', 113.1999969),
 (3309, 'Joint Offset', 113.1999969),
 (3309, 'Joint Offset', 115.40000149999999),
 (3309, 'Root', 115.40000149999999),
 (3309, 'Joint Offset', 118.0)]

In [65]:
colocated/total

0.5583239255428076

In [69]:
"""
Most frequent defect in colocated zone
"""
defects = {'Fracture':0, 'Crack':0, 'Joint Offset':0, 'Root':0, 'Deposit':0, 'Broken':0}

for zone in zones:
    if len(zone) > 1:
        for defect in zone:
            defects[defect[1]] += 1 

In [70]:
defects

{'Fracture': 5781,
 'Crack': 5426,
 'Joint Offset': 1103,
 'Root': 4243,
 'Deposit': 3285,
 'Broken': 1428}

In [67]:
defects

{'Fracture': 8625,
 'Crack': 9233,
 'Joint Offset': 1943,
 'Root': 7943,
 'Deposit': 8287,
 'Broken': 2058}

In [71]:
print(5781/8625)
print(5426/9233)
print(1103/1943)
print(4243/7943)
print(3285/8287)
print(1428/2058)


0.6702608695652174
0.5876746452940539
0.5676788471435924
0.5341810399093542
0.39640400627488837
0.6938775510204082


In [22]:
df_temp = df_cond[df_cond['InspectionID'].isin([6])]
df_temp = df_temp.sort_values(by=["Distance"])
indices = df_temp.index
df_temp.head(10)

,ConditionID,InspectionID,Distance,Counter,PACP_Code,Continuous,Value_1st_Dimension,Value_2nd_Dimension,Value_Percent,Joint,Clock_At_From,Clock_To,Remarks
55,56,6,9.600000,196.0,Deposit,NaN,NaN,NaN,5.0,True,7.0,NaN,NaN
56,57,6,13.700000,271.0,Deposit,S01,NaN,NaN,5.0,False,7.0,5.0,wanders
57,58,6,21.100000,377.0,Fracture,NaN,NaN,NaN,0.0,True,12.0,8.0,NaN
58,59,6,37.099998,486.0,Deposit,NaN,NaN,NaN,5.0,True,7.0,NaN,NaN
59,60,6,75.500000,783.0,Fracture,NaN,NaN,NaN,0.0,True,1.0,NaN,NaN
60,61,6,75.500000,838.0,Broken,NaN,NaN,NaN,0.0,True,7.0,8.0,NaN
62,63,6,112.800003,1106.0,Deposit,NaN,NaN,NaN,5.0,True,7.0,NaN,NaN
64,65,6,195.199997,1560.0,Deposit,NaN,NaN,NaN,5.0,True,7.0,NaN,NaN
65,66,6,236.600006,1765.0,Deposit,F01,NaN,NaN,5.0,False,7.0,5.0,wanders


In [13]:
df_temp = df_cond[df_cond['InspectionID'].isin([6])]
df_temp = df_temp.sort_values(by=["Distance"])
indices = df_temp.index
df_temp.head(10)

,ConditionID,InspectionID,Distance,Counter,PACP_Code,Continuous,Value_1st_Dimension,Value_2nd_Dimension,Value_Percent,Joint,Clock_At_From,Clock_To,Remarks
55,56,6,9.600000,196.0,Deposit,NaN,NaN,NaN,5.0,True,7.0,NaN,NaN
56,57,6,13.700000,271.0,Deposit,S01,NaN,NaN,5.0,False,7.0,5.0,wanders
57,58,6,21.100000,377.0,Fracture,NaN,NaN,NaN,0.0,True,12.0,8.0,NaN
58,59,6,37.099998,486.0,Deposit,NaN,NaN,NaN,5.0,True,7.0,NaN,NaN
59,60,6,75.500000,783.0,Fracture,NaN,NaN,NaN,0.0,True,1.0,NaN,NaN
60,61,6,75.500000,838.0,Broken,NaN,NaN,NaN,0.0,True,7.0,8.0,NaN
62,63,6,112.800003,1106.0,Deposit,NaN,NaN,NaN,5.0,True,7.0,NaN,NaN
64,65,6,195.199997,1560.0,Deposit,NaN,NaN,NaN,5.0,True,7.0,NaN,NaN
65,66,6,236.600006,1765.0,Deposit,F01,NaN,NaN,5.0,False,7.0,5.0,wanders


In [116]:
print(df_temp.at[21, 'PACP_Code'])

Deposit


In [36]:
"""
Make a database with external factors for econometric analysis
"""
# Read the inspections database
df_insp = pd.read_csv('Inspections_Hazen_Sawyer.csv')

# Columns that we want to keep
cols = ['InspectionID', 'Height', 'Down_Rim_to_Invert', 'Up_Rim_to_Invert', 'Height', 'Total_Length', 'Location_Code', 'Location_Details', 'Material', 'Weather', 'Street']

# Defect pairs to keep
keep_defects = ['Fracture', 'Crack', 'Tap', 'Manhole', 'Joint Offset', 'Root', 'Deposit', 'Broken']

df_insp = create_econometric_database(df_insp, cols, keep_defects)

df_insp.head()

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-